In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pvlib
from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain

In [3]:
import matplotlib.pyplot as plt

In [4]:
from parallel_run import *
from utils import *

In [7]:
PATH = Path('../data')

In [8]:
filename = PATH/'DHTMY_SAM_E_0KGKSU.csv'
meta, tmy_data = read_tmy(filename,  coerce_year=2018)

In [9]:
meta

{'Source': 'ExpSolar',
 'Location ID': '00000',
 'City': 'S1',
 'State': 'NA',
 'Country': 'Chile',
 'Latitude': -22.1689,
 'Longitude': -69.4219,
 'Time Zone': -4,
 'Elevation': 1163.0}

In [10]:
tmy_data.head()

,ghi,dni,dhi,temp_air,temp_dew,humidity,air_pressure,wind_speed,wind_dir,snow
datetime,,,,,,,,,,
2018-01-01 00:00:00-04:00,0.0,0.0,0.0,16.7,16.7,81.2,884.9,2.9,2.3,0
2018-01-01 01:00:00-04:00,0.0,0.0,0.0,16.1,16.1,77.0,885.0,1.9,22.3,0
2018-01-01 02:00:00-04:00,0.0,0.0,0.0,15.0,15.0,75.6,885.1,1.4,31.8,0
2018-01-01 03:00:00-04:00,0.0,0.0,0.0,14.7,14.7,74.7,885.1,0.3,38.9,0
2018-01-01 04:00:00-04:00,0.0,0.0,0.0,13.6,13.6,76.6,885.1,1.1,43.2,0


In [13]:
times = tmy_data.index
coordinates = (meta['Latitude'], 
               meta['Longitude'], 
               meta['Time Zone'], 
               meta['Elevation'],
               'Maria Elena')

In [14]:
location = Location(*coordinates)

In [15]:
location.get_solarposition(times=times).head()

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
datetime,,,,,,
2018-01-01 00:00:00-04:00,133.733485,133.733485,-43.733485,-43.733485,193.132186,-3.404951
2018-01-01 01:00:00-04:00,134.597476,134.597476,-44.597476,-44.597476,173.892446,-3.424553
2018-01-01 02:00:00-04:00,130.925969,130.925969,-40.925969,-40.925969,155.732051,-3.444146
2018-01-01 03:00:00-04:00,123.599970,123.599970,-33.599970,-33.599970,140.999548,-3.463729
2018-01-01 04:00:00-04:00,113.826480,113.826480,-23.826480,-23.826480,129.865373,-3.483303


In [16]:
# get the module and inverter specifications from SAM
sandia_modules = pvlib.pvsystem.retrieve_sam('CECmod')
sapm_inverters = pvlib.pvsystem.retrieve_sam('SandiaInverter')

# module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
# inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_']

In [17]:
sapm_inverters.T.head()

,Vac,Paco,Pdco,Vdco,Pso,C0,C1,C2,C3,Pnt,Vdcmax,Idcmax,Mppt_low,Mppt_high
ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_,208.0,250.0,259.522050,40.242603,1.771614,-0.000025,-0.000090,0.000669,-0.018900,0.02,65.0,10.00000,20.0,50.0
ABB__MICRO_0_25_I_OUTD_US_208__208V__208V__CEC_2018_,208.0,250.0,259.589000,40.000000,2.089610,-0.000041,-0.000091,0.000494,-0.013171,0.02,50.0,6.48971,30.0,50.0
ABB__MICRO_0_25_I_OUTD_US_240_240V__CEC_2014_,240.0,250.0,259.552697,39.982246,1.931194,-0.000027,-0.000158,0.001480,-0.034600,0.05,65.0,10.00000,20.0,50.0
ABB__MICRO_0_25_I_OUTD_US_240__240V__240V__CEC_2018_,240.0,250.0,259.492000,40.000000,2.240410,-0.000039,-0.000132,0.002418,-0.014926,0.05,50.0,6.48730,30.0,50.0
ABB__MICRO_0_3_I_OUTD_US_208_208V__CEC_2014_,208.0,300.0,311.714554,40.227111,1.971053,-0.000036,-0.000256,-0.000833,-0.039100,0.02,65.0,10.00000,30.0,50.0


In [18]:
for l in list(sapm_inverters.columns):
    if 'memc' in l.lower() and '315' in l.lower():
        print(l)

In [19]:
for l in list(sandia_modules.columns):
    if 'memc' in l.lower() and '315' in l.lower():
        print(l)

MEMC_Singapore_MEMC_M315BZC_3Y
MEMC_Singapore_MEMC_Q315ACC_34
MEMC_Singapore_MEMC_Q315ACC_35
MEMC_Singapore_MEMC_Q315ACC_36
MEMC_Singapore_MEMC_Q315AEC_34
MEMC_Singapore_MEMC_Q315AEC_35
MEMC_Singapore_MEMC_Q315AEC_36
MEMC_Singapore_MEMC_Q315AMC_34
MEMC_Singapore_MEMC_Q315AMC_35
MEMC_Singapore_MEMC_Q315AMC_36
MEMC_Singapore_MEMC_Q315BZC_3Y
MEMC_Singapore_SE_F315BZC_3Y
MEMC_Singapore_SE_M315BZC_3Y


Pretty close to what we want

In [20]:
module = sandia_modules['MEMC_Singapore_MEMC_M315BZC_3Y']; module

BIPV                  N
Date          1/14/2013
T_NOCT             47.1
A_c               1.956
N_s                  72
I_sc_ref           9.07
V_oc_ref          45.68
I_mp_ref           8.64
V_mp_ref           36.7
alpha_sc       0.006349
beta_oc        -0.16262
a_ref            1.9843
I_L_ref           9.162
I_o_ref         9.2e-10
R_s               0.377
R_sh_ref        4310.23
Adjust            10.28
gamma_r           -0.48
Version          NRELv1
PTC               280.5
Technology    Mono-c-Si
Name: MEMC_Singapore_MEMC_M315BZC_3Y, dtype: object

In [22]:
system = PVSystem(module_parameters=module,
                  modules_per_string=18,
                  strings_per_inverter=250,
                  albedo=0.35)

In [23]:
mc = ModelChain(system, location,
                dcmodel='desoto',
                acmodel='snlinverter',
                orientation_strategy='None')

ValueError: could not infer AC model from system.inverter_parameters. Check system.inverter_parameters or explicitly set the model with the ac_model kwarg.